# Write data to cache

This notebook is meant to be used together with [Read data from cache](./read_data_from_cache.ipynb) to demonstate the use of the datasets cache.

First we setup a simple experiment. This is copied from another notebook and can be ignored in this context.

In [1]:
%matplotlib notebook
import time

import numpy as np

import qcodes as qc
from qcodes.dataset import Measurement, initialise_database, load_or_create_experiment
from qcodes.instrument_drivers.mock_instruments import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Jens-Work\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Jens-Work\.qcodes\logs\200924-21124-qcodes.log


In [2]:
# preparatory mocking of physical setup

dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrumentWithMeasurement('dmm', setter_instr=dac)

station = qc.Station(dmm, dac)

In [3]:
initialise_database()
exp = load_or_create_experiment(experiment_name='dataset_cache_test',
                          sample_name="no sample")

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 499.56it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 499.62it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 332.83it/s]


Now we are ready to run an experiment. Once this experiment is running, take note of the id of the run (also accessible via ``dataset.captured_run_id``) created and open the [Read data from cache](./read_data_from_cache.ipynb) notebook and use there this id.  After 20 sec this notebook will start writing actual data to the dataset.

In [6]:
# And then run an experiment

meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone

meas.write_period = 2


with meas.run() as datasaver:
    time.sleep(20)
    # While sleeping here start loader. From load_cached_notebook.ipynb
    # this is done by loading this new run via ``captured_run_id`` printed when the measurement starts
    print("done sleeping")
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))
        # flush so this always works
        datasaver.flush_data_to_database(block=True)
        time.sleep(0.1)

    dataset = datasaver.dataset  # convenient to have for plotting

Starting experimental run with id: 170. 
done sleeping
